Import libraries

In [18]:
import torch
import cv2
%reload_ext autoreload
%autoreload 2
# this is the main library used (sits on top of PyTorch)
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from sklearn import metrics

Set up data: PATH should lead to the data folder with subfolders train, valid, and test. Train and valid should each have subfolders "yes" and "no". For this to run, you need something in these "yes" and "no" folders, though it won't affect the predictions--just put in some images from your test data.

In [19]:
# where the data is
PATH = "data/example_data"
# using resnet architecture
arch = resnet34
# size of square image in pixels
sz = 44
# transforms used on training data
transforms_up_down = [RandomScale(sz,1.2),RandomRotate(1)]
tfms = tfms_from_model(arch,sz,crop_type = CropType.NO,aug_tfms=transforms_up_down)
# data: comes from PATH, used tfms on training data, bs of 8 for training data, test data located in test folder
data = ImageClassifierData.from_paths(PATH,tfms=tfms,bs=8,test_name='test')
# load in pretraine
state = torch.load('saved_model.pkl',map_location=torch.device('cpu')) # remove map_location parameter if on GPU
learn2 = ConvLearner.pretrained(arch,data,precompute=False)
learn2.model.load_state_dict(state)

<All keys matched successfully>

In [20]:
log_preds_test = learn2.predict(is_test=True)
preds_test = np.argmax(log_preds_test,axis=1)
probs_test = np.exp(log_preds_test[:,1])

In [21]:
test_names = np.empty_like(data.test_ds.fnames)
for i in range(len(data.test_ds.fnames)):
    test_names[i] = data.test_ds.fnames[i]
    #temp = data.test_ds.fnames[i]
    #matchobj = re.search('.*im.*',temp)
    #test_names[i] = matchobj.group()
test_df = pd.DataFrame(data = test_names,columns = ['image_number'])
test_df['prediction'] = preds_test
test_df['probability'] = probs_test

In [22]:
test_df

,image_number,prediction,probability
0,test/new_img_350.jpg,0,0.000214
1,test/new_img_436.jpg,0,0.000277
2,test/new_img_422.jpg,0,0.000005
3,test/new_img_344.jpg,1,0.973887
4,test/new_img_378.jpg,0,0.002214
...,...,...,...
1194,test/new_img_361.jpg,0,0.001094
1195,test/new_img_407.jpg,0,0.035961
1196,test/new_img_413.jpg,0,0.000799
1197,test/new_img_375.jpg,0,0.000464
